In [50]:
import csv
import numpy as np

In [51]:
def read_and_rewrite(data_from):
    indics_per_ego = {}
    
    with open('../Data/%s.csv' % data_from, 'r') as to_read:
        csv_r = csv.reader(to_read, delimiter = ';')
        list_indics = csv_r.next()[1:]
        values_per_indic = {indic : [] for indic in list_indics}
        for line in csv_r:
            indics_per_ego[line[0]] = {}
            for i in range(len(list_indics)):
                this_indic_value = float(line[i+1])
                indics_per_ego[line[0]][list_indics[i]] = this_indic_value
                values_per_indic[list_indics[i]].append(this_indic_value)
        
    average_indics = {indic : np.average(values_per_indic[indic]) for indic in list_indics}

    with open('../Data/%s_normalized.csv' % data_from, 'w') as to_write:
        csv_w = csv.writer(to_write, delimiter = ';')
        en_tete = ['ego'] + list_indics
        csv_w.writerow(en_tete)
        
        for ego in indics_per_ego:
            row = [ego] + [indics_per_ego[ego][indic]/float(average_indics[indic]) for indic in list_indics]
            csv_w.writerow(row)

In [52]:
def calcul_representativity_global(matrix, list_indics):
    
    matrix_2 = [[float(x) for x in matrix[ego]] for ego in matrix]
    #print matrix_2
    
    sum_indics = [sum([ego[i] for ego in matrix_2]) for i in range(len(matrix_2[0]))]
    sum_egos = [sum(ego) for ego in matrix_2]
    
    sum_all = sum(sum_egos)
    global_repr = [sum_indics[i]/float(sum_all) for i in range(len(sum_indics))]
     
    return global_repr

def calcul_representativity_local(patterns_list):
    total_pat = sum(patterns_list)
    return [x/float(total_pat) for x in patterns_list]

In [59]:
def read_and_rewrite_repr(data_from):
    indics_per_ego, local_repr_per_ego = {}, {}
    
    with open('../Data/%s.csv' % data_from, 'r') as to_read:
        csv_r = csv.reader(to_read, delimiter = ';')
        list_indics = csv_r.next()[1:]
        
        for line in csv_r:
            ego_name = line[0]
            indics_per_ego[ego_name] = []
            for i in range(len(list_indics)):
                indics_per_ego[ego_name].append(int(line[i+1]))
            
            try:
                local_repr_per_ego[ego_name] = calcul_representativity_local(indics_per_ego[ego_name])
            except:
                print ego_name
        
    global_repr = calcul_representativity_global(indics_per_ego, list_indics)
    
    with open('../Data/%s_normalized.csv' % data_from, 'w') as to_write:
        csv_w = csv.writer(to_write, delimiter = ';')
        en_tete = ['ego'] + list_indics
        csv_w.writerow(en_tete)
        
        for ego in indics_per_ego:
            row = [ego] + [local_repr_per_ego[ego][i]/global_repr[i] for i in range(len(global_repr))]
            csv_w.writerow(row)

In [61]:
read_and_rewrite_repr('caen_k5')